# Lecture 13
# Introduction to PySpark
### April 28, 2020






Part of this lecture is based on the material by: https://nyu-cds.github.io/python-bigdata/

Some articles - relevant:
- [Getting Started with Spark (in Python)](https://districtdatalabs.silvrback.com/getting-started-with-spark-in-python)
- [A Hands-on Introduction to MapReduce in Python](https://zettadatanet.wordpress.com/2015/04/04/a-hands-on-introduction-to-mapreduce-in-python)
- [Spark Dataframes and MLlib](http://www.techpoweredmath.com/spark-dataframes-mllib-tutorial/)
- [Hadoop with Spark](https://blog.cloudera.com/blog/2015/09/how-to-prepare-your-apache-hadoop-cluster-for-pyspark-jobs/)

For this class you will need to have spark and pyspark installed. Instructions can be found in:
- [MacOS](https://medium.freecodecamp.org/installing-scala-and-apache-spark-on-mac-os-837ae57d283f)
- [Linux](https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f)
- [Windows](http://changhsinlee.com/install-pyspark-windows-jupyter/)

To use python3 we will need to install `Python3, Scala and Apache Spark`

for example on mac os: 

```
brew update
brew install python3
brew install scala
brew install apache-spark```



---

Need to set a var: 

```shell
export PYSPARK_PYTHON=python3
```

---

---

### Distributed computing framework needs to solve two problems: 
1. how to distribute data 
2. how to distribute computation

---


## Hadoop vs Spark
Hadoop and Spark are both projects from [Apache](http://apache.org) Software Foundation. 



- __Hadoop__ software library is a framework that allows for the **distributed processing** of **large data sets across clusters of computers** using simple programming models. It is designed to scale up from single servers to thousands of machines. The library itself is designed to detect and handle failures at the application layer, so delivering a highly-available service on top of a cluster of computers, each of which may be **prone to failures**.  


- __Spark__ differs from Hadoop in several aspects, but mainly in that it **processes data in RAM** using a concept known as a **Resilient Distributed Dataset (RDD)**. Spark is structured around Spark Core, the engine that drives the scheduling, optimizations, and RDD abstraction.


- RDDs are essentially a programming abstraction that represents a read-only collection of objects that are partitioned across machines. RDDs are fault tolerant and are accessed via parallel operations.  


<img src="./figs/interactive_operations_on_mapreduce.jpg" alt="Drawing" style="width: 300px;"/>
<img src="./figs/interactive_operations_on_spark_rdd.jpg" alt="Drawing" style="width: 400px;"/>


- **Because RDDs can be cached in memory, Spark is extremely effective at iterative applications**, where the data is being reused throughout the course of an algorithm. Most machine learning and optimization algorithms are iterative, making Spark an extremely effective tool for data science. 

<img src="./figs/logistic-regression-hadoopXspark.png" alt="Drawing" style="width: 200px;"/>  


- The **Spark library** contains a lot of the **application elements** that have found their way into most Big Data applications including support for SQL-like querying, machine learning and graph algorithms, and even support for live streaming data.

<img src="./figs/sparkstack.png" alt="Drawing" style="width: 300px;"/>


---
## The concept of MapReduce
MapReduce is a framework for processing large data sets in a distributed fashion. The _core idea_ behind MapReduce is **mapping your data set into a collection of (key, value) pairs**, and then **reducing over all pairs with the same key**.

The overall concept is simple, but is actually quite expressive when you consider that:
- Almost all data can be mapped into (key, value) pairs somehow, 
- Keys and values may be of any type: strings, integers, dummy types, and, of course, (key, value) pairs themselves


**Example:** Counting the number of occurrences of words in a text is considered as the “Hello world!” equivalent of MapReduce. A classical way to write such a program is presented in the python script below (you will need [pg2701.txt](https://nyu-cds.github.io/python-bigdata/files/pg2701.txt)).

In [1]:
#####
# count words simpler version
##### 

import re

# remove any non-words and split line into separate words
# finally, convert all words to lowercase
def splitter(line):
    line = re.sub(r'^\W+|\W+$', '', line)
    return map(str.lower, re.split(r'\W+', line))
    
cnt = {}
try:
    in_file = open('pg2701.txt', 'r')

    for line in in_file:
        for word in splitter(line):
            #word = word.lower()
            cnt[word] = cnt.get(word, 0) + 1

    in_file.close()

except IOError:
    print("error performing file operation")
else:
    _max = max(cnt.keys(), key=lambda w: cnt[w])
    print("max: '%s' = %d" % (_max, cnt[_max]))
    print("conter: ", cnt)

max: 'the' = 14620
conter:  {'the': 14620, 'project': 91, 'gutenberg': 94, 'ebook': 10, 'of': 6732, 'moby': 89, 'dick': 89, 'or': 797, 'whale': 1233, 'by': 1226, 'herman': 4, 'melville': 4, '': 3235, 'this': 1443, 'is': 1751, 'for': 1646, 'use': 49, 'anyone': 6, 'anywhere': 16, 'at': 1335, 'no': 596, 'cost': 4, 'and': 6502, 'with': 1770, 'almost': 197, 'restrictions': 2, 'whatsoever': 7, 'you': 963, 'may': 255, 'copy': 19, 'it': 2536, 'give': 90, 'away': 186, 're': 18, 'under': 126, 'terms': 33, 'license': 18, 'included': 14, 'online': 4, 'www': 6, 'org': 13, 'title': 8, 'author': 9, 'last': 278, 'updated': 2, 'january': 1, '3': 15, '2009': 1, 'posting': 1, 'date': 5, 'december': 5, '25': 3, '2008': 1, '2701': 4, 'release': 1, 'june': 3, '2001': 2, 'language': 7, 'english': 50, 'start': 37, 'produced': 9, 'daniel': 4, 'lazarus': 7, 'jonesey': 2, 'original': 23, 'transcriber': 1, 's': 1816, 'notes': 4, 'text': 6, 'a': 4799, 'combination': 2, 'etexts': 1, 'one': 925, 'from': 1105, 'now':

### The number of words processed per second

The issue with the code above is that **the performance degrades as the size of the dictionary grows**. As shown on the diagram below, the number of words processed per second diminishes when the size of the dictionary reaches the size of the processor data cache (note that if the cache is structured in several layers of different speeds, the processing speed will decrease each time the dictionary reaches the size of a layer).

A larger decrease in processing speed occurs when the dictionary reaches the size of the computer’s RAM.

Eventually, if the dictionary continues to grow, it will exceed the capacity of the swap space and an exception will be raised.
<img src="./figs/performance.png" alt="Drawing" style="width: 500px;"/>

### Using MapReduce approach
The main advantage of the MapReduce approach is that it **does not require a central data structure** so the memory issues that occur with the simplistic approch are avoided.

#### MapReduce consists of 3 steps:

- A **mapping** step that produces intermediate results and associates them with an output key;
- A **shuffling** step that groups intermediate results associated with the same output key; and
- A **reducing** step that processes groups of intermediate results with the same output key.
<img src="./figs/mapreducewc.png" alt="Drawing" style="width: 700px;"/>

#### Mapping
The idea is to apply a function to each element of a list and collect the result, essentially the same as the Python _map method_. Using the Python _map_ in the example above would simply result in reading the whole file into memory before performing the map operation, and  this would be no better than the original version. 

<img src="./figs/mapping.png" alt="Drawing" style="width: 400px;"/>

Instead, we must perform the map operation using a temporary file (that we will use later), as follows:

In [2]:
import re

# remove any non-words and split lines into separate words
# finally, convert all words to lowercase
def splitter(line):
    line = re.sub(r'^\W+|\W+$', '', line)
    return map(str.lower, re.split(r'\W+', line))
    
input_file = 'pg2701.txt'
map_file   = 'pg2701.txt.map'

# Implement our mapping function
import re
cnt = {}
try:
    in_file = open(input_file, 'r')
    out_file = open(map_file, 'w')

    for line in in_file:
        for word in splitter(line):
            out_file.write(word + "\t1\n") # Separate key and value with 'tab'
            
    in_file.close()
    out_file.close()

except IOError:
    print("error performing file operation")

Notice that the file is not read into memory as a whole. The code just read a line at a time, map the words in the line, then read the next line, etc. The resulting file contains lines that look like this:
```shell
moby    1
dick    1
or  1
the 1
whale   1
:
:
```

#### Shuffling (grouping by the key)
The shuffling step consists of grouping all the intermediate values that have the same key. In  word count example, we want to sort the intermediate key/value pairs on their keys.
<img src="./figs/shuffling.png" alt="Drawing" style="width: 200px;"/>

Code below resort to the intermediate _pg2701.txt.map_ filed created in the previous code to avoid memory issues when dealing with larger data sets.

In [ ]:
map_file = 'pg2701.txt.map'
sorted_map_file = 'pg2701.txt.map.sorted'

def build_index(filename):
    index = []
    f = open(filename)
    while True:
        offset = f.tell()
        line = f.readline()
        if not line:
            break
        length = len(line)
        col = line.split('\t')[0].strip()
        index.append((col, offset, length))
        print((col, offset, length))
    f.close()
    # here the index is sorted
    index.sort()
    return index

try:
    index = build_index(map_file)
    in_file = open(map_file, 'r')
    out_file = open(sorted_map_file, 'w')
    
    for col, offset, length in index:
        in_file.seek(offset)
        out_file.write(in_file.read(length))
    in_file.close()
    out_file.close()

except IOError:
    print("error performing file operation")

('the', 0, 6)
('project', 6, 10)
('gutenberg', 16, 12)
('ebook', 28, 8)
('of', 36, 5)
('moby', 41, 7)
('dick', 48, 7)
('or', 55, 5)
('the', 60, 6)
('whale', 66, 8)
('by', 74, 5)
('herman', 79, 9)
('melville', 88, 11)
('', 99, 3)
('this', 102, 7)
('ebook', 109, 8)
('is', 117, 5)
('for', 122, 6)
('the', 128, 6)
('use', 134, 6)
('of', 140, 5)
('anyone', 145, 9)
('anywhere', 154, 11)
('at', 165, 5)
('no', 170, 5)
('cost', 175, 7)
('and', 182, 6)
('with', 188, 7)
('almost', 195, 9)
('no', 204, 5)
('restrictions', 209, 15)
('whatsoever', 224, 13)
('you', 237, 6)
('may', 243, 6)
('copy', 249, 7)
('it', 256, 5)
('give', 261, 7)
('it', 268, 5)
('away', 273, 7)
('or', 280, 5)
('re', 285, 5)
('use', 290, 6)
('it', 296, 5)
('under', 301, 8)
('the', 309, 6)
('terms', 315, 8)
('of', 323, 5)
('the', 328, 6)
('project', 334, 10)
('gutenberg', 344, 12)
('license', 356, 10)
('included', 366, 11)
('with', 377, 7)
('this', 384, 7)
('ebook', 391, 8)
('or', 399, 5)
('online', 404, 9)
('at', 413, 5)
('www', 

('small', 17541, 8)
('sized', 17549, 8)
('one', 17557, 6)
('', 17563, 3)
('the', 17566, 6)
('aorta', 17572, 8)
('of', 17580, 5)
('a', 17585, 4)
('whale', 17589, 8)
('is', 17597, 5)
('larger', 17602, 9)
('in', 17611, 5)
('the', 17616, 6)
('bore', 17622, 7)
('than', 17629, 7)
('the', 17636, 6)
('main', 17642, 7)
('pipe', 17649, 7)
('of', 17656, 5)
('the', 17661, 6)
('water', 17667, 8)
('works', 17675, 8)
('at', 17683, 5)
('london', 17688, 9)
('bridge', 17697, 9)
('and', 17706, 6)
('the', 17712, 6)
('water', 17718, 8)
('roaring', 17726, 10)
('in', 17736, 5)
('its', 17741, 6)
('passage', 17747, 10)
('through', 17757, 10)
('that', 17767, 7)
('pipe', 17774, 7)
('is', 17781, 5)
('inferior', 17786, 11)
('in', 17797, 5)
('impetus', 17802, 10)
('and', 17812, 6)
('velocity', 17818, 11)
('to', 17829, 5)
('the', 17834, 6)
('blood', 17840, 8)
('gushing', 17848, 10)
('from', 17858, 7)
('the', 17865, 6)
('whale', 17871, 8)
('s', 17879, 4)
('heart', 17883, 8)
('paley', 17891, 8)
('s', 17899, 4)
('theol

('of', 32465, 5)
('the', 32470, 6)
('land', 32476, 7)
('loitering', 32483, 12)
('under', 32495, 8)
('the', 32503, 6)
('shady', 32509, 8)
('lee', 32517, 6)
('of', 32523, 5)
('yonder', 32528, 9)
('warehouses', 32537, 13)
('will', 32550, 7)
('not', 32557, 6)
('suffice', 32563, 10)
('no', 32573, 5)
('they', 32578, 7)
('must', 32585, 7)
('get', 32592, 6)
('just', 32598, 7)
('as', 32605, 5)
('nigh', 32610, 7)
('the', 32617, 6)
('water', 32623, 8)
('as', 32631, 5)
('they', 32636, 7)
('possibly', 32643, 11)
('can', 32654, 6)
('without', 32660, 10)
('falling', 32670, 10)
('in', 32680, 5)
('and', 32685, 6)
('there', 32691, 8)
('they', 32699, 7)
('stand', 32706, 8)
('miles', 32714, 8)
('of', 32722, 5)
('them', 32727, 7)
('leagues', 32734, 10)
('inlanders', 32744, 12)
('all', 32756, 6)
('they', 32762, 7)
('come', 32769, 7)
('from', 32776, 7)
('lanes', 32783, 8)
('and', 32791, 6)
('alleys', 32797, 9)
('streets', 32806, 10)
('and', 32816, 6)
('avenues', 32822, 10)
('north', 32832, 8)
('east', 32840,

('of', 52091, 5)
('misty', 52096, 8)
('spray', 52104, 8)
('and', 52112, 6)
('these', 52118, 8)
('words', 52126, 8)
('underneath', 52134, 13)
('the', 52147, 6)
('spouter', 52153, 10)
('inn', 52163, 6)
('peter', 52169, 8)
('coffin', 52177, 9)
('', 52186, 3)
('coffin', 52189, 9)
('spouter', 52198, 10)
('rather', 52208, 9)
('ominous', 52217, 10)
('in', 52227, 5)
('that', 52232, 7)
('particular', 52239, 13)
('connexion', 52252, 12)
('thought', 52264, 10)
('i', 52274, 4)
('but', 52278, 6)
('it', 52284, 5)
('is', 52289, 5)
('a', 52294, 4)
('common', 52298, 9)
('name', 52307, 7)
('in', 52314, 5)
('nantucket', 52319, 12)
('they', 52331, 7)
('say', 52338, 6)
('and', 52344, 6)
('i', 52350, 4)
('suppose', 52354, 10)
('this', 52364, 7)
('peter', 52371, 8)
('here', 52379, 7)
('is', 52386, 5)
('an', 52391, 5)
('emigrant', 52396, 11)
('from', 52407, 7)
('there', 52414, 8)
('as', 52422, 5)
('the', 52427, 6)
('light', 52433, 8)
('looked', 52441, 9)
('so', 52450, 5)
('dim', 52455, 6)
('and', 52461, 6)
('

('the', 71692, 6)
('more', 71698, 7)
('i', 71705, 4)
('abominated', 71709, 13)
('the', 71722, 6)
('thought', 71728, 10)
('of', 71738, 5)
('sleeping', 71743, 11)
('with', 71754, 7)
('him', 71761, 6)
('it', 71767, 5)
('was', 71772, 6)
('fair', 71778, 7)
('to', 71785, 5)
('presume', 71790, 10)
('that', 71800, 7)
('being', 71807, 8)
('a', 71815, 4)
('harpooneer', 71819, 13)
('his', 71832, 6)
('linen', 71838, 8)
('or', 71846, 5)
('woollen', 71851, 10)
('as', 71861, 5)
('the', 71866, 6)
('case', 71872, 7)
('might', 71879, 8)
('be', 71887, 5)
('would', 71892, 8)
('not', 71900, 6)
('be', 71906, 5)
('of', 71911, 5)
('the', 71916, 6)
('tidiest', 71922, 10)
('certainly', 71932, 12)
('none', 71944, 7)
('of', 71951, 5)
('the', 71956, 6)
('finest', 71962, 9)
('i', 71971, 4)
('began', 71975, 8)
('to', 71983, 5)
('twitch', 71988, 9)
('all', 71997, 6)
('over', 72003, 7)
('besides', 72010, 10)
('it', 72020, 5)
('was', 72025, 6)
('getting', 72031, 10)
('late', 72041, 7)
('and', 72048, 6)
('my', 72054, 5)

...

('', 499068, 3)
('', 499071, 3)
('', 499074, 3)
('chapter', 499077, 10)
('39', 499087, 5)
('first', 499092, 8)
('night', 499100, 8)
('watch', 499108, 8)
('', 499116, 3)
('fore', 499119, 7)
('top', 499126, 6)
('', 499132, 3)
('stubb', 499135, 8)
('solus', 499143, 8)
('and', 499151, 6)
('mending', 499157, 10)
('a', 499167, 4)
('brace', 499171, 8)
('', 499179, 3)
('', 499182, 3)
('ha', 499185, 5)
('ha', 499190, 5)
('ha', 499195, 5)
('ha', 499200, 5)
('hem', 499205, 6)
('clear', 499211, 8)
('my', 499219, 5)
('throat', 499224, 9)
('i', 499233, 4)
('ve', 499237, 5)
('been', 499242, 7)
('thinking', 499249, 11)
('over', 499260, 7)
('it', 499267, 5)
('ever', 499272, 7)
('since', 499279, 8)
('and', 499287, 6)
('that', 499293, 7)
('ha', 499300, 5)
('ha', 499305, 5)
('s', 499310, 4)
('the', 499314, 6)
('final', 499320, 8)
('consequence', 499328, 14)
('why', 499342, 6)
('so', 499348, 5)
('because', 499353, 10)
('a', 499363, 4)
('laugh', 499367, 8)
('s', 499375, 4)
('the', 499379, 6)
('wisest', 4993

('chance', 516040, 9)
('caught', 516049, 9)
('sight', 516058, 8)
('of', 516066, 5)
('him', 516071, 6)
('in', 516077, 5)
('the', 516082, 6)
('beginning', 516088, 12)
('of', 516100, 5)
('the', 516105, 6)
('thing', 516111, 8)
('they', 516119, 7)
('had', 516126, 6)
('every', 516132, 8)
('one', 516140, 6)
('of', 516146, 5)
('them', 516151, 7)
('almost', 516158, 9)
('as', 516167, 5)
('boldly', 516172, 9)
('and', 516181, 6)
('fearlessly', 516187, 13)
('lowered', 516200, 10)
('for', 516210, 6)
('him', 516216, 6)
('as', 516222, 5)
('for', 516227, 6)
('any', 516233, 6)
('other', 516239, 8)
('whale', 516247, 8)
('of', 516255, 5)
('that', 516260, 7)
('species', 516267, 10)
('but', 516277, 6)
('at', 516283, 5)
('length', 516288, 9)
('such', 516297, 7)
('calamities', 516304, 13)
('did', 516317, 6)
('ensue', 516323, 8)
('in', 516331, 5)
('these', 516336, 8)
('assaults', 516344, 11)
('not', 516355, 6)
('restricted', 516361, 13)
('to', 516374, 5)
('sprained', 516379, 11)
('wrists', 516390, 9)
('and', 5

('earth', 536401, 8)
('his', 536409, 6)
('root', 536415, 7)
('of', 536422, 5)
('grandeur', 536427, 11)
('his', 536438, 6)
('whole', 536444, 8)
('awful', 536452, 8)
('essence', 536460, 10)
('sits', 536470, 7)
('in', 536477, 5)
('bearded', 536482, 10)
('state', 536492, 8)
('an', 536500, 5)
('antique', 536505, 10)
('buried', 536515, 9)
('beneath', 536524, 10)
('antiquities', 536534, 14)
('and', 536548, 6)
('throned', 536554, 10)
('on', 536564, 5)
('torsoes', 536569, 10)
('so', 536579, 5)
('with', 536584, 7)
('a', 536591, 4)
('broken', 536595, 9)
('throne', 536604, 9)
('the', 536613, 6)
('great', 536619, 8)
('gods', 536627, 7)
('mock', 536634, 7)
('that', 536641, 7)
('captive', 536648, 10)
('king', 536658, 7)
('so', 536665, 5)
('like', 536670, 7)
('a', 536677, 4)
('caryatid', 536681, 11)
('he', 536692, 5)
('patient', 536697, 10)
('sits', 536707, 7)
('upholding', 536714, 12)
('on', 536726, 5)
('his', 536731, 6)
('frozen', 536737, 9)
('brow', 536746, 7)
('the', 536753, 6)
('piled', 536759, 8

('us', 557130, 5)
('add', 557135, 6)
('that', 557141, 7)
('even', 557148, 7)
('the', 557155, 6)
('king', 557161, 7)
('of', 557168, 5)
('terrors', 557173, 10)
('when', 557183, 7)
('personified', 557190, 14)
('by', 557204, 5)
('the', 557209, 6)
('evangelist', 557215, 13)
('rides', 557228, 8)
('on', 557236, 5)
('his', 557241, 6)
('pallid', 557247, 9)
('horse', 557256, 8)
('', 557264, 3)
('therefore', 557267, 12)
('in', 557279, 5)
('his', 557284, 6)
('other', 557290, 8)
('moods', 557298, 8)
('symbolize', 557306, 12)
('whatever', 557318, 11)
('grand', 557329, 8)
('or', 557337, 5)
('gracious', 557342, 11)
('thing', 557353, 8)
('he', 557361, 5)
('will', 557366, 7)
('by', 557373, 5)
('whiteness', 557378, 12)
('no', 557390, 5)
('man', 557395, 6)
('can', 557401, 6)
('deny', 557407, 7)
('that', 557414, 7)
('in', 557421, 5)
('its', 557426, 6)
('profoundest', 557432, 14)
('idealized', 557446, 12)
('significance', 557458, 15)
('it', 557473, 5)
('calls', 557478, 8)
('up', 557486, 5)
('a', 557491, 4)


('strictly', 577623, 11)
('confined', 577634, 11)
('to', 577645, 5)
('its', 577650, 6)
('own', 577656, 6)
('unavoidable', 577662, 14)
('straight', 577676, 11)
('wake', 577687, 7)
('yet', 577694, 6)
('the', 577700, 6)
('arbitrary', 577706, 12)
('vein', 577718, 7)
('in', 577725, 5)
('which', 577730, 8)
('at', 577738, 5)
('these', 577743, 8)
('times', 577751, 8)
('he', 577759, 5)
('is', 577764, 5)
('said', 577769, 7)
('to', 577776, 5)
('swim', 577781, 7)
('generally', 577788, 12)
('embraces', 577800, 11)
('some', 577811, 7)
('few', 577818, 6)
('miles', 577824, 8)
('in', 577832, 5)
('width', 577837, 8)
('more', 577845, 7)
('or', 577852, 5)
('less', 577857, 7)
('as', 577864, 5)
('the', 577869, 6)
('vein', 577875, 7)
('is', 577882, 5)
('presumed', 577887, 11)
('to', 577898, 5)
('expand', 577903, 9)
('or', 577912, 5)
('contract', 577917, 11)
('but', 577928, 6)
('never', 577934, 8)
('exceeds', 577942, 10)
('the', 577952, 6)
('visual', 577958, 9)
('sweep', 577967, 8)
('from', 577975, 7)
('the',

One might argue that the __build_index()__ method is loading the whole **pg2701.txt.map**  file in memory, what is similar to the simpler version above. 

However, the __shuffling__ version can be split into many independent parallel tasks. 

#### Reduce
**The reduce step just counts the number of values with the same key.** Now that the different values are ordered by keys (i.e., the different words are listed in alphabetic order), it becomes easy to count the number of times each key occurs.
<img src="./figs/reducing.png" alt="Drawing" style="width: 300px;"/>

In [ ]:
# previously defined: 
# sorted_map_file = 'pg2701.txt.map.sorted'
# in_file: map_file = 'pg2701.txt.map'

previous = None
# M is global and keeps the current most frequent word and its frequency
M = [None, 0] 

def checkmax(key, cnt):
    global m, M
    if M[1] < cnt:
        M[0] = key
        M[1] = cnt
        
try:
    in_file = open(sorted_map_file, 'r')
    for line in in_file:
        key, value = line.split('\t')
        if key != previous:
            if previous is not None:
                checkmax(previous, cnt)
            previous = key
            cnt = 0
            
        cnt += int(value)
        
    checkmax(previous, cnt)
    in_file.close()
    
except IOError:
    print("error performing file operation")
    
print("max: '%s' = %d" % (M[0], M[1]))

---
## Spark Overview
A Spark program typically follows a simple paradigm:
- __A driver is the main program__.
- __One or more workers run code sent by the driver__ on their partitions of the RDD, which is distributed across the cluster.
- __Results are sent back to the driver__ for aggregation or compilation.

RDD is a Resilient Distributed Data set, which is essentially a distributed collection of items.

---

When Spark runs a closure (function) on a worker, any variables used in the closure are copied to that node, but are maintained within the local scope of that closure.

---
Spark provides __two types of shared variables__ that can be interacted with by all workers in a restricted fashion:

- __Broadcast variables__ are distributed to all workers, but are read-only. These variables can be used as lookup tables or stopword lists.
- __Accumulators__ are variables that workers can “add” to using associative operations and are typically used as counters.

---

### Spark Execution

- Spark applications are run as independent sets of processes, **coordinated by a SparkContext in the driver program**. 

- The context will connect to some cluster manager which allocates system resources.

- Each worker in the cluster is managed by an executor, which is in turn managed by the SparkContext. 

- The executor manages computation as well as storage and caching on each machine.

<img src="./figs/cluster.png" alt="Drawing" style="width: 400px;"/>

What is important to note is that:
- Application code is sent from the driver to the executors, which specify the context and the various tasks to be run.
- The executors communicate back and forth with the driver for data sharing or for interaction.
- Drivers are key participants in Spark jobs, and therefore, they should be on the same network as the cluster.

### MapReduce with Spark
To start using Spark, we have to create an RDD. The SparkContext provides a number of methods to do this. We will use the **textFile** method, which reads a file an creates an RDD of strings, one for each line in the file.

In [ ]:
%%writefile "test_spark.py"
from pyspark import SparkContext

#master "local" and appname = "wordcount"
sc = SparkContext(master="local", appName="wordcount")

# creating a RDD from pg2701.txt
text = sc.textFile('pg2701.txt')

print(text.getNumPartitions())

# Running this program display the first 10 entries in the RDD:
print("RDD:", text.take(10))

---

Execute with ```spark-submit file_name```


---

---

We use the same splitter function we used previously to split lines correctly. The **flatMap** method applies the function to all elements of the RDD and flattens the results into a single list of words.

In [ ]:
%%writefile "test_spark_counts.py"
from pyspark import SparkContext
from operator import add # Required for reduceByKey
import re

# remove any non-words and split lines into separate words
# finally, convert all words to lowercase
def splitter(line):
    line = re.sub(r'^\W+|\W+$', '', line)
    return map(str.lower, re.split(r'\W+', line))

if __name__ == '__main__':
    sc = SparkContext("local", "wordcount")
    
    # creating a RDD from pg2701.txt
    text = sc.textFile('pg2701.txt')
    
    # creating a list of words in the RDD
    words = text.flatMap(splitter)
#     print("words:", words.take(10))
    
    # mapping the list of words in a tuple
    # make (key, value) pairs
    words_mapped = words.map(lambda w: (w, 1))
    
    #sort them
    sorted_map = words_mapped.sortByKey()
    
    #reduce by key i.e. count how many of the words have a given key
    counts = sorted_map.reduceByKey(add)
    
    print("counts: ", counts.take(500))
    
    

---
### Number of primes


Spark also provides the parallelize method which distributes a local Python collection to form an RDD (obviously a cluster is required to obtain true parallelism.)

---

In [ ]:
%%writefile "test_spark_primes.py"
from pyspark import SparkContext

def isprime(n):
    """
    check if integer n is a prime
    """
    # make sure n is a positive integer
    n = abs(int(n))
    
    # 0 and 1 are not primes
    if n < 2: return False
    
    # 2 is the only even prime number
    if n == 2: return True
    
    # all other even numbers are not primes
    if not n & 1: return False
    
    # range starts with 3 and only needs to go up the square root of n
    # for all odd numbers
    for x in range(3, int(n ** 0.5) + 1, 2):
        if n % x == 0: return False
    
    return True
    
if __name__ == '__main__':
    sc = SparkContext("local", "primes")
    # Create an RDD of numbers from 0 to 1,000,000
    
    nums = sc.parallelize(range(1_000_000))
    
    # Compute the number of primes in the RDD
    print("counts: ", nums.filter(isprime).count())